# Практическая часть 3
Иерархическая кластеризация и дендрограммы

## Блок 0. Подготовка окружения

Импортировать библиотеки для работы с данными, расстояниями и иерархической кластеризацией.
Настроить отображение таблиц.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

%matplotlib inline

pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

## Блок 1. Набор данных Iris для кластеризации

### 1.1. Загрузка данных и формирование матрицы признаков

Загрузить набор данных Iris.
Добавить человекочитаемое название вида и выделить матрицу числовых признаков для кластеризации.

In [ ]:
iris = datasets.load_iris(as_frame=True)
df_iris = iris.frame.copy()
df_iris["species"] = df_iris["target"].map(dict(enumerate(iris.target_names)))

X = df_iris[iris.feature_names]
df_iris.head()

### 1.2. Проверка данных

Проверить форму матрицы признаков и отсутствие пропусков.

In [ ]:
print("Форма X:", X.shape)
print("Пропуски по столбцам:")
print(X.isna().sum())

## Блок 2. Матрица расстояний (евклидово расстояние)

### 2.1. Вычисление попарных расстояний

Для всех пар объектов вычислить евклидово расстояние по четырём признакам.
Использовать `pdist` для вычисления расстояний и `squareform` для получения квадратной матрицы расстояний.

In [ ]:
distances_condensed = pdist(X.values, metric="euclidean")
distance_matrix = squareform(distances_condensed)

distance_matrix.shape

### 2.2. Просмотр части матрицы расстояний

Представить матрицу расстояний в виде таблицы и вывести её небольшой фрагмент (например, первые 10×10).

In [ ]:
dist_df = pd.DataFrame(
    distance_matrix,
    index=df_iris.index,
    columns=df_iris.index
)

dist_df.iloc[:10, :10]

## Блок 3. Иерархическая кластеризация

### 3.1. Построение иерархии кластеров (агломеративный метод)

Выполнить агломеративную иерархическую кластеризацию на основе матрицы расстояний.
Использовать метод связки `ward` (или другой по выбору).

In [ ]:
Z = linkage(distances_condensed, method="ward")
Z[:10]

### 3.2. Построение дендрограммы

Построить дендрограмму, показывающую структуру кластеров и расстояния между ними.
В качестве подписей листьев использовать названия видов (species).

In [ ]:
plt.figure(figsize=(12, 6))
dendrogram(
    Z,
    labels=df_iris["species"].values,
    leaf_rotation=90,
    leaf_font_size=8
)
plt.title("Иерархическая кластеризация Iris (дендрограмма)")
plt.xlabel("Объекты (виды)")
plt.ylabel("Евклидово расстояние")
plt.tight_layout()
plt.show()

## Блок 4. Выбор числа кластеров и разбиение

### 4.1. Получение кластеров по числу групп

Разбить дендрограмму на заданное число кластеров (например, 3 кластера).
Добавить номера кластеров в исходный DataFrame.

In [ ]:
n_clusters = 3
cluster_labels_hc = fcluster(Z, n_clusters, criterion="maxclust")

df_iris["cluster_hc"] = cluster_labels_hc
df_iris[["species", "cluster_hc"]].head()

### 4.2. Сводная таблица «виды ↔ кластеры»

Построить перекрёстную таблицу:
строки — реальные виды (species),
столцы — кластеры иерархической кластеризации.

In [ ]:
ct_hc = pd.crosstab(df_iris["species"], df_iris["cluster_hc"])
ct_hc

## Блок 5. Визуализация кластеров на диаграмме рассеяния

### 5.1. Отображение кластеров в пространстве признаков

Визуализировать кластеры, найденные иерархической кластеризацией, на диаграмме рассеяния:
по осям — длина и ширина лепестка,
цветом — номер кластера.

In [ ]:
plt.figure(figsize=(6, 5))
plt.scatter(
    df_iris["petal length (cm)"],
    df_iris["petal width (cm)"],
    c=df_iris["cluster_hc"],
    alpha=0.8
)
plt.xlabel("Petal length (cm)")
plt.ylabel("Petal width (cm)")
plt.title("Иерархическая кластеризация (кластеры на признаках лепестков)")
plt.show()

### 5.2. Сравнение с реальными классами

Для сравнения построить диаграмму рассеяния с теми же осями, но раскрасить точки по реальным видам (species).
Сравнить визуально, насколько кластеры совпадают с видами.

In [ ]:
plt.figure(figsize=(6, 5))
for species_name, sub_df in df_iris.groupby("species"):
    plt.scatter(
        sub_df["petal length (cm)"],
        sub_df["petal width (cm)"],
        label=species_name,
        alpha=0.8
    )
plt.xlabel("Petal length (cm)")
plt.ylabel("Petal width (cm)")
plt.title("Реальные виды Iris на признаках лепестков")
plt.legend()
plt.show()

## Блок 6. (Опционально) Разные методы связки

По желанию: повторить иерархическую кластеризацию с другими методами связки (`single`, `complete`, `average`) и сравнить дендрограммы и результаты разбиения.

In [ ]:
Z_complete = linkage(distances_condensed, method="complete")

plt.figure(figsize=(12, 6))
dendrogram(
    Z_complete,
    labels=df_iris["species"].values,
    leaf_rotation=90,
    leaf_font_size=8
)
plt.title("Иерархическая кластеризация (complete linkage)")
plt.xlabel("Объекты (виды)")
plt.ylabel("Евклидово расстояние")
plt.tight_layout()
plt.show()